In [1]:
import torch
import matplotlib.pyplot as plt
import ESN
import time
import numpy as np
import json

In [2]:
torch.set_default_device("cuda:0")
torch.set_default_dtype(torch.double)

In [3]:
fn = input("set file name : ")

set file name :  a


In [13]:
## calculate memory capacity

with open('./experiments/settings/'+fn+'_s.txt', 'r') as fp:
    setting = json.load(fp)

u = torch.load('./experiments/inputs/'+fn+'_i.pt')
Xwo = torch.load('./experiments/datamatrices/'+fn+'_d.pt')

N = setting["Nodes"]
dim = setting["input dim"]
if setting["identical Win"]:
    maxtau = int(N*2) 
else : maxtau =  int(N/dim *2) 

""" tweak maxtau here """
##maxtau=int(N*10)
##

st = time.time()
#mfs = ESN.MC_cSVD_old(u_sym, Xwo, maxtau)
#mfs = ESN.MC_cSVD(u_sym, Xwo, maxtau)
mfs_raw, mfs_lin, mfs_rev, sur_value = ESN.MC_cSVD(u_sym, Xwo, maxtau,ret_all=True)
mfs = mfs_rev

print(r"mc calc time :%.3f s"%(time.time()-st))
print("total MC :",float(torch.sum(mfs)))
print(float(sur_value))


#torch.save(mfs,f"./experiments/mfs/{fn}_mf.pt")
#torch.save(mfs_raw,f"./experiments/mfs/{fn}_raw.pt")
#torch.save(sur_value,f"./experiments/mfs/{fn}_sur.pt")


mc calc time : 0.04304385185241699
total MC : 23.358996903637674
0.005013125079519831


In [3]:
#
fn="50N_tanh_10000T"

In [4]:
## load data
with open('./experiments/settings/'+fn+'_s.txt', 'r') as fp:
    setting = json.load(fp)
u_sym = torch.load('./experiments/inputs/'+fn+'_i.pt')
Xwo = torch.load('./experiments/datamatrices/'+fn+'_d.pt')

Two = setting["Two"]


In [6]:
"""  TEMPORARY CELL  """

## set max delay, degree
maxdd=[[3,50]]

st = time.time()
target_info = ESN.make_targets(u_sym,maxdd,Two=Two)
print(r"target create time:%.3f s"%(time.time()-st))
torch.save(target_info,f"./experiments/target_info/{fn}_3rd_ti.pt")

#st = time.time()
#ti = torch.load('./experiments/target_info/'+fn+'_7th_ti.pt')
#print("load time:",time.time()-st)

basis table creation:0.002 s
[tensor([[-0.3361, -0.8083, -0.9399,  ..., -0.2002, -0.0868, -0.6585],
        [ 0.8918, -0.3361, -0.8083,  ..., -0.3137, -0.2002, -0.0868],
        [ 0.8221,  0.8918, -0.3361,  ..., -0.1119, -0.3137, -0.2002],
        ...,
        [ 0.7825, -0.2839,  0.5726,  ..., -0.5989,  0.8230,  0.4655],
        [-0.1091,  0.7825, -0.2839,  ..., -0.2057, -0.5989,  0.8230],
        [ 0.7708, -0.1091,  0.7825,  ...,  0.4180, -0.2057, -0.5989]],
       device='cuda:0'), tensor([[-0.3306,  0.4801,  0.8250,  ..., -0.4399, -0.4887,  0.1504],
        [ 0.6931, -0.3306,  0.4801,  ..., -0.3524, -0.4399, -0.4887],
        [ 0.5139,  0.6931, -0.3306,  ..., -0.4812, -0.3524, -0.4399],
        ...,
        [ 0.4184, -0.3791, -0.0081,  ...,  0.0381,  0.5160, -0.1749],
        [-0.4821,  0.4184, -0.3791,  ..., -0.4365,  0.0381,  0.5160],
        [ 0.3911, -0.4821,  0.4184,  ..., -0.2379, -0.4365,  0.0381]],
       device='cuda:0'), tensor([[ 0.4092, -0.1079, -0.6658,  ...,  0.2803,  

In [7]:
"""  TEMPORARY CELL  """
#ti = ti.slice(0,4000)
ti=target_info
st = time.time()
raw,lin,rev,sur = ESN.calc_capacity(Xwo,ti.tar_f,ret_all=True)
print(r"ipc calc time :%.3f s"%(time.time()-st))
capacities = rev


ipc=ESN.IPC(capacities,ti.maxdelay,ti.degree,ti.maxddset,ti.in_dim)
#torch.save(ipc,f"./experiments/ipcs/{fn}_7th_ipc.pt")
torch.save(ipc,f"./experiments/ipcs/{fn}_3rd_ipc.pt")


ipc calc time :0.140 s


In [8]:

torch.save(raw,f"./experiments/ipcs/{fn}_3rd_raw.pt")
torch.save(sur,f"./experiments/ipcs/{fn}_3rd_sur.pt")

In [5]:
## calculate ipc

## create target functions

## set max delay, degree
maxdd=[[2,10],[3,25],[4,10],[5,20]]

st = time.time()
target_info = ESN.make_targets(u_sym,maxdd,Two=Two)
print(r"target create time:%.3f s"%(time.time()-st))
torch.save(target_info,f"./experiments/target_info/{fn}_ti.pt")



7 degree:116280 target functions
target create time:1802.085 s


In [19]:
## calculate capacities
ti = torch.load('./experiments/target_info/'+fn+'_ti.pt')

st = time.time()
raw,lin,rev,sur = ESN.calc_capacity(Xwo,ti.tar_f,ret_all=True)
print(r"ipc calc time :%.3f s"%(time.time()-st))
capacities = rev

ipc=ESN.IPC(capacities,ti.maxdelay,ti.degree,ti.maxddset,ti.in_dim)
torch.save(ipc,f"./experiments/ipcs/{fn}_ipc.pt")
#torch.save(capacities,f"./experiments/ipcs/{fn}_c.pt")


ipc : 0.041225433349609375


In [21]:
mfs = torch.load(f"./experiments/mfs/{fn}_mf.pt")
ipc = torch.load(f"./experiments/ipcs/{fn}_ipc.pt")

dgrs = torch.tensor(ipc.degree)

totMC=float(torch.sum(mfs))
totCapacity = totMC+float(torch.sum(capacities))
print("totoal capacity:",totCapacity)
print("MC:",totMC)



c_2nd=torch.sum(capacities[torch.argwhere(dgrs==2)])
c_3rd=torch.sum(capacities[torch.argwhere(dgrs==3)])
c_4th=torch.sum(capacities[torch.argwhere(dgrs==4)])
c_5th=torch.sum(capacities[torch.argwhere(dgrs==5)])
print("2nd order ipc:",float(c_2nd))
print("3rd order ipc:",float(c_3rd))
print("4th order ipc:",float(c_4th))
print("5th order ipc:",float(c_5th))


average ipc value: 0.0007974275874987815
totoal capacity: 60.19935401849388
MC: 23.358996903637674
2nd order ipc: -0.03119727354854423
3rd order ipc: 24.114016373215414
4th order ipc: 0.05783274524159601
5th order ipc: 12.699705269947742
